# Determination of the suitable chromatography

This part of the code allows to find the most suitable chromatography based on the physical-chemical properties of the molecules saved on a data frame.
To do this the following dependencies has tp be installed:

In [21]:
import pandas as pd

Pandas will create a data frame from a dictionary as follows:

In [30]:
data = {
    'Mixture': ['Caf', 'Ace', 'Asp'],
    'Boiling Point': [178, 332.7, 246],
    'XLogP': [-0.07, -1.33, 0.07],
    'pKa': [[14], [3.02], [1.08, 9.13]],
    'MolecularWeight': [194, 204, 133],
}

df = pd.DataFrame(data)
print(df)

  Mixture  Boiling Point  XLogP           pKa  MolecularWeight
0     Caf          178.0  -0.07          [14]              194
1     Ace          332.7  -1.33        [3.02]              204
2     Asp          246.0   0.07  [1.08, 9.13]              133


As can be seen in the second column it is indicated the names of all the molecules from the mixture. Then the properties used by the function are the boiling point in [°C], the XlogP, the pKa and the molecular weight in [g/mol].

Following an example of the functionality of the function:

In [31]:
 def det_chromato(df):
    boiling_temps = [temp for temp in df['Boiling Point'] if temp is not None]
    if boiling_temps and max(boiling_temps) <= 250:
        Chromato_type = 'GC'
        eluent_nature = 'gas'
        proposed_pH = None
    else:
        molar_masses = [mass for mass in df['MolecularWeight'] if mass is not None]
        max_molar_mass = max(molar_masses) if molar_masses else None

        min_pKa = float('inf')  # Initialize min_pKa with a large value
        max_pKa = float('-inf')  # Initialize max_pKa with a small value
        for pKa_entry in df['pKa']:
            if isinstance(pKa_entry, list):
                for pKa_value in pKa_entry:
                    if pKa_value is not None:
                        min_pKa = min(pKa_value, min_pKa)
                        max_pKa = max(pKa_value, max_pKa)
            else:
                if pKa_entry is not None:
                    min_pKa = min(pKa_entry, min_pKa)
                    max_pKa = max(pKa_entry, max_pKa)
        
        if min_pKa == float('inf'):
            min_pKa = None
        if max_pKa == float('-inf'):
            max_pKa = None

        logPs = [logP for logP in df['XLogP'] if logP is not None]
        max_logP = max(logPs) if logPs else None
        min_logP = min(logPs) if logPs else None

        if max_molar_mass is not None and max_molar_mass <= 2000:
            if max_logP is not None and max_logP < 0:
                proposed_pH = max_pKa + 2 if max_pKa is not None else None
                if proposed_pH is not None and 3 <= proposed_pH <= 11:
                    Chromato_type = 'IC'
                    eluent_nature = 'aqueous'
                else:
                    Chromato_type = 'HPLC'
                    eluent_nature = 'organic or hydro-organic'
                    proposed_pH = min_pKa + 2 if min_pKa is not None else None
            else:
                Chromato_type = 'HPLC'
                if min_logP is not None and -2 <= min_logP <= 0:
                    eluent_nature = 'organic or hydro-organic'
                    if min_logP >= 0:
                        Chromato_type += ' on normal stationary phase'
                    else:
                        Chromato_type += ' on reverse stationary phase using C18 column'
                else:
                    eluent_nature = 'organic or hydro-organic'
                    Chromato_type += ' on normal stationary phase'
                proposed_pH = min_pKa + 2 if min_pKa is not None else None
        else:
            if max_logP is not None and max_logP < 0:
                Chromato_type = 'HPLC on reverse stationary phase'
                eluent_nature = 'organic or hydro-organic'
                proposed_pH = min_pKa + 2 if min_pKa is not None else None
            else:
                if max_logP is not None and max_logP > 0:
                    Chromato_type = 'SEC on gel permeation with a hydrophobe organic polymer stationary phase'
                    eluent_nature = 'organic solvent'
                else:
                    Chromato_type = 'SEC on gel filtration with a polyhydroxylated hydrophile polymer stationary phase'
                    eluent_nature = 'aqueous'
                proposed_pH = min_pKa + 2 if min_pKa is not None else None
    
    return Chromato_type, eluent_nature, proposed_pH

Mixture_chromato_type, eluent_nature, proposed_pH = det_chromato(df)

The function will look up column per column in the data frame and search the maximum or the minimum of each fisical or chemical properties. Based on that it start to predict the best chromatogrphy to perform:

The GC will be performed if the max **boiling temperature** of the mixture is under 250 °C, this will ensure the high volatility of the mixture.

The **molar weight** will indicate if the steric exclusion chromatography is allowed or not. It is suitable for heavy molecules, thus for the compounds in the mixture with a mass larger than 2000 g/mol.

The choice between high pressure liquid chromatography (HPLC) and ionic chromatography (IC) is determined by the **XlogP**, wich is an indicator of the hydrophilicity of the molecules. Negative values for the **XlogP** indicates that the molecule is likely to dissolve in water (thus hydrophile), positive values indicates that the molecule is more likely to dissolve in organic solvents (thus hydrofobe). Therefore the more hydrophile is the molecule the more likely it will ionize, hence the suitable chromatography will be IC. 

In the liquid chromatography such as HPLC or IC the pH of the eluent is an important parameter which can lead to a good separation. All the molecules have to be stable at this pH otherwise they can degrade and the separation will not be good. to ensure the stability of the molecules, the function propose a pH wich is 2 values above the minimum **pKa** of the mixture compunds.

In [32]:
print(f"The advisable chromatography type is: {Mixture_chromato_type}")
print(f"Eluent nature: {eluent_nature}")
if proposed_pH is not None:
    print(f"Proposed pH for the eluent: {proposed_pH}")

The advisable chromatography type is: HPLC on reverse stationary phase using C18 column
Eluent nature: organic or hydro-organic
Proposed pH for the eluent: 3.08
